In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install wxee rasterio

In [ ]:
import xarray as xr
import ee
import itertools
import pickle
import pandas as pd
import wxee
import rasterio

In [ ]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

In [ ]:
soil_wc = xr.open_dataset('/content/drive/MyDrive/w/soil_water_content_1979001.nc')
soil_wc

In [ ]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file="/content/drive/MyDrive/w/private-key.json"
)

ee.Initialize(credentials)

In [ ]:
shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])

CDL = ee.ImageCollection("USDA/NASS/CDL")

# Crop Mask selection

cropland = CDL.filterDate('2022-01-01','2022-12-31').select('cropland').filterBounds(shape)

# cropland = cropland.set("system:time_start", ee.Date("2022"))
arr_cropland = cropland.wx.to_xarray(scale=10000, crs='EPSG:4326')

xds_repr_match_land_use = arr_cropland.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.mode)
xds_repr_match_land_use

In [ ]:
xds_repr_match_land_use.cropland[0,150:158,200:210]

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/w/cdl_table.xlsx", sheet_name='Sheet2')
df